In [1]:
# Import libraries
import pandas as pd
import requests
from datetime import date

# Headers for API Request
nba_header_data  = {
    'Connection': 'keep-alive',
    'Accept': 'application/json, text/plain, */*',
    'x-nba-stats-token': 'true',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36',
    'x-nba-stats-origin': 'stats',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-Mode': 'cors',
    'Referer': 'https://stats.nba.com/',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'en-US,en;q=0.9',
}

In [3]:
# Functions
def get_nba_games_today(nba_header_data):
    
    ### Function to scrape NBA API for today's games
    ## Args: 
    # nba_header_data (dict): headers for NBA API request
    ## Returns (df): dataframe of nba_games_today with today's games
    ###
    
    # Get system date
    today = date.today()
    
    # Paste into url
    url = 'https://stats.nba.com/stats/scoreboardv3?GameDate=' + str(today) + '&LeagueID=00'
    
    # Send request
    r = requests.get(url, headers=nba_header_data)
    
    # Get JSON response
    resp = r.json()
    
    # Set json resp columns to grab
    nba_json_schedule_cols = ['gameId', 'gameEt', 'awayTeam.teamId', 'awayTeam.teamTricode', 'awayTeam.teamName',
                            'homeTeam.teamId', 'homeTeam.teamTricode', 'homeTeam.teamName']

    # Normalize json and select columns
    nba_games_today = pd.json_normalize(resp["scoreboard"]['games'])[nba_json_schedule_cols]

    # Create awayteamName and homeTeamName
    nba_games_today['awayTeamName'] = nba_games_today['awayTeam.teamTricode'] + ' ' + nba_games_today['awayTeam.teamName']
    nba_games_today['homeTeamName'] = nba_games_today['homeTeam.teamTricode'] + ' ' + nba_games_today['homeTeam.teamName']

    # Rename some columns
    nba_games_today.rename({'awayTeam.teamId': 'awayTeamId', 'awayTeam.teamTricode': 'awayTeamSlug', 
                            'homeTeam.teamId': 'homeTeamId', 'homeTeam.teamTricode': 'homeTeamSlug'}, axis=1, inplace=True)

    # Set columns to select
    nba_schedule_cols = ['gameId', 'gameEt', 'awayTeamId', 'awayTeamSlug', 'awayTeamName',
                        'homeTeamId', 'homeTeamSlug', 'homeTeamName']

    # Select columns
    nba_games_today = nba_games_today[nba_schedule_cols]

    ## --- INSERT/UPDATE SQL --- ##
    # - Update all other columns where game id = gameId - #
    
    return nba_games_today 

In [4]:
# Get today's schedule
nba_games_today = get_nba_games_today(nba_header_data)